In [17]:
import sys
sys.path.append("..")

import numpy as np
import scipy.misc as smp
import matplotlib.pyplot as plt
import cPickle as pickle

import theano
import theano.tensor as T

import scae_destin.datasets as ds
from scae_destin.fflayers import ReLULayer
from scae_destin.fflayers import SoftmaxLayer
from scae_destin.convnet import ReLUConvLayer
from scae_destin.convnet import SigmoidConvLayer
from scae_destin.model import ConvAutoEncoder
from scae_destin.convnet import MaxPooling
from scae_destin.convnet import Flattener
from scae_destin.model import FeedForward
from scae_destin.optimize import gd_updates
from scae_destin.cost import mean_square_cost
from scae_destin.cost import categorical_cross_entropy_cost
from scae_destin.cost import L2_regularization

In [2]:
n_epochs=1
batch_size=100
nkerns=100

Xtr, Ytr=ds.load_CIFAR_batch("../cifar-10-batches-py/data_batch_1")

Xtr=np.mean(Xtr, 3)
Xtrain=Xtr.reshape(Xtr.shape[0], Xtr.shape[1]*Xtr.shape[2])/255.0


train_set_x, train_set_y=ds.shared_dataset((Xtrain, Ytr))

n_train_batches=train_set_x.get_value(borrow=True).shape[0]/batch_size

print "[MESSAGE] The data is loaded"


[MESSAGE] The data is loaded


In [3]:

################################## FIRST LAYER #######################################

X=T.matrix("data")
y=T.ivector("label")
idx=T.lscalar()
corruption_level=T.fscalar()

images=X.reshape((batch_size, 1, 32, 32))

layer_0_en=ReLUConvLayer(filter_size=(7,7),
                         num_filters=50,
                         num_channels=1,
                         fm_size=(32,32),
                         batch_size=batch_size)
                                                  
layer_0_de=SigmoidConvLayer(filter_size=(7,7),
                            num_filters=1,
                            num_channels=50,
                            fm_size=(26,26),
                            batch_size=batch_size,
                            border_mode="full")
                         
layer_1_en=ReLUConvLayer(filter_size=(5,5),
                         num_filters=50,
                         num_channels=50,
                         fm_size=(26,26),
                         batch_size=batch_size)
                                                   
layer_1_de=SigmoidConvLayer(filter_size=(5,5),
                            num_filters=50,
                            num_channels=50,
                            fm_size=(22,22),
                            batch_size=batch_size,
                            border_mode="full")

layer_2_en=ReLUConvLayer(filter_size=(5,5),
                         num_filters=50,
                         num_channels=50,
                         fm_size=(22,22),
                         batch_size=batch_size)
                                                   
layer_2_de=SigmoidConvLayer(filter_size=(5,5),
                            num_filters=50,
                            num_channels=50,
                            fm_size=(18,18),
                            batch_size=batch_size,
                            border_mode="full")

layer_3_en=ReLUConvLayer(filter_size=(3,3),
                         num_filters=50,
                         num_channels=50,
                         fm_size=(18,18),
                         batch_size=batch_size)
                                                   
layer_3_de=SigmoidConvLayer(filter_size=(3,3),
                            num_filters=50,
                            num_channels=50,
                            fm_size=(16,16),
                            batch_size=batch_size,
                            border_mode="full")

model_0=ConvAutoEncoder(layers=[layer_0_en, layer_0_de])
out_0=model_0.fprop(images, corruption_level=corruption_level)
cost_0=mean_square_cost(out_0[-1], images)+L2_regularization(model_0.params, 0.005)
updates_0=gd_updates(cost=cost_0, params=model_0.params, method="sgd", learning_rate=0.1)

model_1=ConvAutoEncoder(layers=[layer_1_en, layer_1_de])
out_1=model_1.fprop(out_0[0], corruption_level=corruption_level)
cost_1=mean_square_cost(out_1[-1], out_0[0])+L2_regularization(model_1.params, 0.005)
updates_1=gd_updates(cost=cost_1, params=model_1.params, method="sgd", learning_rate=0.1)

model_2=ConvAutoEncoder(layers=[layer_2_en, layer_2_de])
out_2=model_2.fprop(out_1[0], corruption_level=corruption_level)
cost_2=mean_square_cost(out_2[-1], out_1[0])+L2_regularization(model_2.params, 0.005)
updates_2=gd_updates(cost=cost_2, params=model_2.params, method="sgd", learning_rate=0.1)

model_3=ConvAutoEncoder(layers=[layer_3_en, layer_3_de])
out_3=model_3.fprop(out_2[0], corruption_level=corruption_level)
cost_3=mean_square_cost(out_3[-1], out_2[0])+L2_regularization(model_3.params, 0.005)
updates_3=gd_updates(cost=cost_3, params=model_3.params, method="sgd", learning_rate=0.1)

train_0=theano.function(inputs=[idx, corruption_level],
                        outputs=[cost_0],
                        updates=updates_0,
                        givens={X: train_set_x[idx * batch_size: (idx + 1) * batch_size]})

train_1=theano.function(inputs=[idx, corruption_level],
                        outputs=[cost_1],
                        updates=updates_1,
                        givens={X: train_set_x[idx * batch_size: (idx + 1) * batch_size]})

train_2=theano.function(inputs=[idx, corruption_level],
                        outputs=[cost_2],
                        updates=updates_2,
                        givens={X: train_set_x[idx * batch_size: (idx + 1) * batch_size]})

train_3=theano.function(inputs=[idx, corruption_level],
                        outputs=[cost_3],
                        updates=updates_3,
                        givens={X: train_set_x[idx * batch_size: (idx + 1) * batch_size]})
                      
print "[MESSAGE] The 4-layer model is built"

[MESSAGE] The 4-layer model is built


In [84]:
layer = layer_0_de

data = np.zeros(((layer.filter_size[0] + 3) * layer.num_filters,(layer.filter_size[1] + 3) * layer.num_channels), dtype=np.float32 )
data[::] = -0.1 #some activations are below 0, to keep borders black set them to -0.1

for filter_index in xrange(layer.num_filters):
    for channel_index in xrange(layer.num_channels):
        arr = layer.filters.get_value()[filter_index][channel_index]
        for p in xrange(layer.filter_size[0]):
            for q in xrange(layer.filter_size[1]):
                data[filter_index*(3+layer.filter_size[1])+p][channel_index*(3+layer.filter_size[1])+q] = arr[p][q]
            
print data.shape
img = smp.toimage(data)
img.save("layer_0_de.png")


(10, 500)


In [26]:
layer_0_de.filter_size

(7, 7)

In [14]:
plt.savefig('plot_layer0_de.png')

In [ ]:
plt.show()

In [23]:
img = smp.toimage( arr )

ValueError: 'arr' does not have a suitable array shape for any mode.

In [20]:
img.save("rawdraw.png")